In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve,roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import roc_auc_score , roc_curve
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

pd.set_option('display.max_rows' , None)
pd.set_option('display.max_columns' , None)

In [ ]:
test = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/test_l0Auv8Q.csv')
train = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Train.csv')
first_health_camp = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/First_Health_Camp_Attended.csv')
second_health_camp = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Second_Health_Camp_Attended.csv')
patient_profile = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Patient_Profile.csv')
third_health_camp = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Third_Health_Camp_Attended.csv')
health_camp = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Health_Camp_Detail.csv')
sub=pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/sample_submmission.csv')

In [ ]:
train.head()

In [ ]:
first_health_camp.head()

In [ ]:
second_health_camp.head()

In [ ]:
third_health_camp.head()

In [ ]:
patient_profile.head()

In [ ]:
health_camp.head()

In [ ]:
train['Registration_Date'].nunique()

In [ ]:
len(train)

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(y=train['Registration_Date'])

In [ ]:
train['Patient_ID'].nunique()

In [ ]:
train[['Var1','Var2','Var3','Var4']].boxplot()

In [ ]:
sns.boxplot(x="variable", y="value", data=pd.melt(train[['Var1','Var2','Var3','Var4']]))

In [ ]:
sns.barplot(x='variable',y='value',data=pd.melt(train[['Var1','Var2','Var3','Var4']]))

In [ ]:
data=pd.melt(train[['Var1','Var2','Var3','Var4']])

In [ ]:
data.head()

In [ ]:
sns.distplot(train['Health_Camp_ID'],kde=False)

In [ ]:
df= pd.merge(train,health_camp,how='left',on=['Health_Camp_ID'])
df_1= pd.merge(df,patient_profile,how='left',on=['Patient_ID'])
df_2=pd.merge(df_1,first_health_camp,how='left',on=['Patient_ID','Health_Camp_ID'])
df_3=pd.merge(df_2,second_health_camp,how='left',on=['Patient_ID', 'Health_Camp_ID'])
df_final=pd.merge(df_3,third_health_camp,how='left',on=['Patient_ID', 'Health_Camp_ID'])

In [ ]:
df_final.head()

In [ ]:
data_1=pd.melt(df_final[['Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared']])

In [ ]:
data_1.head()

In [ ]:
sns.countplot(data_1[data_1['value']==1]['variable'])

In [ ]:
df_final['Category1'].value_counts()

In [ ]:
sns.countplot(df_final['Category1'])

In [ ]:
sns.countplot(df_final['Category2'])

In [ ]:
sns.distplot(df_final['Category3'],kde=False)

In [ ]:
a=df_final[df_final['Income']!='None']

In [ ]:
sns.distplot(a['Income'],kde=False)

In [ ]:
sns.countplot(df_final['Income'])

In [ ]:
sns.countplot(a['Income'])

In [ ]:
plt.figure(figsize=(25,7))
sns.countplot(df_final['Employer_Category'])

In [ ]:
sns.distplot(df_final['Donation'],kde=False)

In [ ]:
sns.countplot(df_final['City_Type'])

In [ ]:
b=df_final[df_final['Age']!='None']

In [ ]:
sns.distplot(b['Age'])

In [ ]:
c=df_final[df_final['Education_Score']!='None']

In [ ]:
sns.distplot(c['Education_Score'])

In [ ]:
df_final.info()

In [ ]:
def change_to_int(r):

    if r == 'None':
        return None
    else:
        return float(r)
df_final['Age'] = df_final['Age'].apply(change_to_int)   
df_final['Education_Score'] = df_final['Education_Score'].apply(change_to_int)
df_final['Income'] = df_final['Income'].apply(change_to_int)

In [ ]:
a = df_final.isnull().sum()/len(df_final)*100
a[a>0]

In [ ]:
df_final['Age'].median()

In [ ]:
df_final.drop(columns = ['Unnamed: 4'] , inplace = True)
df_final['Age'].fillna(44.0 , inplace = True)
df_final['Employer_Category'].fillna('Unemployed' , inplace = True)
df_final['Income'].fillna(0 , inplace = True)
df_final['Education_Score'].fillna(0 , inplace = True)

In [ ]:
df_final.groupby(['Income'])['City_Type'].value_counts()

In [ ]:
def fill_city(x):
    City = x[0]
    Income = x[1]
    if pd.isnull(City):
        if Income ==0:
            return 'H'
        else:
            return 'B'
        
    else:
        return City

df_final['City_Type'] = df_final[['City_Type' , 'Income']].apply(fill_city,axis=1)    

In [ ]:
df_final.isnull().sum()

In [ ]:
def find_outcomes(x):
    first = x[0]
    second = x[1]
    third = x[2]
    if pd.isnull(first) == False:
        return 1
    elif pd.isnull(second) == False:
        return 1
    elif pd.isnull(third) == False:
        if third > 0:
            return 1
        else:
            return 0
    else:
        return 0
        

In [ ]:
df_final['Target'] = df_final[['Health_Score' , 'Health Score' , 'Number_of_stall_visited']].apply(find_outcomes,axis =1)

In [ ]:
df_final.head()

In [ ]:
a = df_final[pd.isnull(df['Registration_Date'])].index 
df_final.drop(a , inplace = True)

In [ ]:
import datetime
from datetime import datetime

def convert_datetime(x):
    
    return datetime.strptime(x ,  '%d-%b-%y')

df_final['Registration_Date'] = df_final['Registration_Date'].apply(convert_datetime)
df_final['Camp_Start_Date'] = df_final['Camp_Start_Date'].apply(convert_datetime)
df_final['First_Interaction'] = df_final['First_Interaction'].apply(convert_datetime)
df_final['Camp_End_Date'] = df_final['Camp_End_Date'].apply(convert_datetime)


In [ ]:
drop_cols=['Camp_Start_Date','Camp_End_Date','Patient_ID','Health_Camp_ID','Donation',
           'Health_Score','Health Score','Number_of_stall_visited','Last_Stall_Visited_Number',
           'Registration_Date' , 'First_Interaction' ]

df_final.drop(columns = drop_cols , inplace = True)

In [ ]:
df_final.head()

In [ ]:
X=df_final.drop('Target',axis=1)
y=df_final['Target']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=101)

In [ ]:
label_cols = ['Category1','Category2']
encoder = LabelEncoder()
for item in label_cols:
    X_train[item] = encoder.fit_transform(X_train[item])
    X_test[item] = encoder.transform(X_test[item])

In [ ]:
dummie_cols=['City_Type','Employer_Category']
X_train=pd.get_dummies(X_train,columns=dummie_cols,drop_first=True)
X_test=pd.get_dummies(X_test,columns=dummie_cols,drop_first=True)

In [ ]:
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
lr_model = LogisticRegression()
lr_model.fit(X_train , y_train)

In [ ]:
pred_lr = lr_model.predict_proba(X_test)[:,-1]

In [ ]:
roc_lr = roc_auc_score(y_test , pred_lr)
roc_lr

In [ ]:
model_rf = RandomForestClassifier(n_estimators=324,random_state=101,criterion='entropy')
model_rf.fit(X_train , y_train)

In [ ]:
pred_rf = model_rf.predict_proba(X_test)[:,-1]

In [ ]:
roc_rf = roc_auc_score(y_test , pred_rf)
roc_rf

In [ ]:
model_xgb = xgb.XGBClassifier()
model_xgb.fit(X_train , y_train)

In [ ]:
pred_xgb = model_xgb.predict_proba(X_test)[:,-1]

In [ ]:
roc_xgb = roc_auc_score(y_test , pred_xgb)
roc_xgb

In [ ]:
model=Sequential()

model.add(Dense(49,activation='relu'))

model.add(Dense(64,activation='relu'))

model.add(Dense(128,activation='relu'))

model.add(Dense(256,activation='relu'))

model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
early_stop=EarlyStopping(monitor='val_accuracy',mode='min',verbose=1,patience=25)

In [ ]:
y_train1=np.array(y_train)
y_test1=np.array(y_test)

In [ ]:
model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test),callbacks=[early_stop])

In [ ]:
pred_learning=model.predict_proba(X_test)

In [ ]:
learning_auc=roc_auc_score(y_test,pred_learning)
learning_auc